In [154]:
from scipy.stats import e
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
from sklearn.preprocessing import normalize
import math

In [155]:
#for f in os.listdir('nns'):
w_dict = {}
nns = np.load('nns.pkl')
for f in os.listdir('embds_pkls'):
    embds = np.load(os.path.join('embds_pkls', f))
    for k, v in embds.items():
        key = k.split('_')[0] + '.png'
        nns[key] = nns[key] / np.linalg.norm(nns[key])
        v = v / np.linalg.norm(v)
        d = np.dot(nns[key], v)
        d = np.arccos(d) / math.pi
        w_dict[k[:-4]] = 1-d
print(w_dict)

{'404922_50': 0.6046431514667605, '957995_43': 0.5019137671461746, '754125_7': 0.528019395227385, '957995_9': 0.604444468852904, '506904_13': 0.6932044277462691, '571071_36': 0.6597228973531097, '571071_14': 0.7119358231173275, '957995_44': 0.6188242937855042, '872210_18': 0.6623038361129805, '957995_12': 0.6480376958845657, '506904_15': 0.671350573450644, '404922_40': 0.6321025989324582, '404926_44': 0.49789659138583164, '404926_34': 0.5245057941363489, '957995_42': 0.6210826199435782, '957995_16': 0.6128511036288511, '404922_4': 0.5612640129200834, '957995_33': 0.6128042409420458, '571071_38': 0.7257570713850545, '957995_8': 0.6262641152844359, '754127_4': 0.6129146623336276, '754120_19': 0.6903144418992029, '754125_50': 0.5083564569172753, '506904_43': 0.5976652404831986, '506904_14': 0.5902747101827771, '754125_57': 0.6178732658345707, '754125_13': 0.5938344529962062, '872210_53': 0.6943461128387844, '404922_9': 0.6519673313870464, '872210_15': 0.6424813369947604, '957995_56': 0.58

In [156]:
class _PSD(object):
    def __init__(self, M, cond=None, rcond=None, lower=True,
                 check_finite=True, allow_singular=True):
        # Compute the symmetric eigendecomposition.
        # Note that eigh takes care of array conversion, chkfinite,
        # and assertion that the matrix is square.
        s, u = scipy.linalg.eigh(M, lower=lower, check_finite=check_finite)

        eps = _eigvalsh_to_eps(s, cond, rcond)
        if np.min(s) < -eps:
            raise ValueError('the input matrix must be positive semidefinite')
        d = s[s > eps]
        if len(d) < len(s) and not allow_singular:
            raise np.linalg.LinAlgError('singular matrix')
        s_pinv = _pinv_1d(s, eps)
        U = np.multiply(u, np.sqrt(s_pinv))

        # Initialize the eagerly precomputed attributes.
        self.rank = len(d)
        self.U = U
        self.log_pdet = np.sum(np.log(d))

        # Initialize an attribute to be lazily computed.
        self._pinv = None

    @property
    def pinv(self):
        if self._pinv is None:
            self._pinv = np.dot(self.U, self.U.T)
        return self._pinv

def _pinv_1d(v, eps=1e-5):
    return np.array([0 if abs(x) <= eps else 1/x for x in v], dtype=float)
    

def _eigvalsh_to_eps(spectrum, cond=None, rcond=None):
    """
    Determine which eigenvalues are "small" given the spectrum.

    This is for compatibility across various linear algebra functions
    that should agree about whether or not a Hermitian matrix is numerically
    singular and what is its numerical matrix rank.
    This is designed to be compatible with scipy.linalg.pinvh.

    Parameters
    ----------
    spectrum : 1d ndarray
        Array of eigenvalues of a Hermitian matrix.
    cond, rcond : float, optional
        Cutoff for small eigenvalues.
        Singular values smaller than rcond * largest_eigenvalue are
        considered zero.
        If None or -1, suitable machine precision is used.

    Returns
    -------
    eps : float
        Magnitude cutoff for numerical negligibility.

    """
    if rcond is not None:
        cond = rcond
    if cond in [None, -1]:
        t = spectrum.dtype.char.lower()
        factor = {'f': 1E3, 'd': 1E6}
        cond = factor[t] * np.finfo(t).eps
    eps = cond * np.max(abs(spectrum))
    return eps

def _process_parameters(dim, mean, cov):
    """
    Infer dimensionality from mean or covariance matrix, ensure that
    mean and covariance are full vector resp. matrix.

    """

    # Try to infer dimensionality
    if dim is None:
        if mean is None:
            if cov is None:
                dim = 1
            else:
                cov = np.asarray(cov, dtype=float)
                if cov.ndim < 2:
                    dim = 1
                else:
                    dim = cov.shape[0]
        else:
            mean = np.asarray(mean, dtype=float)
            dim = mean.size
    else:
        if not np.isscalar(dim):
            raise ValueError("Dimension of random variable must be a scalar.")

    # Check input sizes and return full arrays for mean and cov if necessary
    if mean is None:
        mean = np.zeros(dim)
    mean = np.asarray(mean, dtype=float)

    if cov is None:
        cov = 1.0
    cov = np.asarray(cov, dtype=float)

    if dim == 1:
        mean.shape = (1,)
        cov.shape = (1, 1)

    if mean.ndim != 1 or mean.shape[0] != dim:
        raise ValueError("Array 'mean' must be a vector of length %d." % dim)
    if cov.ndim == 0:
        cov = cov * np.eye(dim)
    elif cov.ndim == 1:
        cov = np.diag(cov)
    elif cov.ndim == 2 and cov.shape != (dim, dim):
        rows, cols = cov.shape
        if rows != cols:
            msg = ("Array 'cov' must be square if it is two dimensional,"
                   " but cov.shape = %s." % str(cov.shape))
        else:
            msg = ("Dimension mismatch: array 'cov' is of shape %s,"
                   " but 'mean' is a vector of length %d.")
            msg = msg % (str(cov.shape), len(mean))
        raise ValueError(msg)
    elif cov.ndim > 2:
        raise ValueError("Array 'cov' must be at most two-dimensional,"
                         " but cov.ndim = %d" % cov.ndim)

    return dim, mean, cov

def _process_quantiles(x, dim):
    """
    Adjust quantiles array so that last axis labels the components of
    each data point.

    """
    x = np.asarray(x, dtype=float)

    if x.ndim == 0:
        x = x[np.newaxis]
    elif x.ndim == 1:
        if dim == 1:
            x = x[:, np.newaxis]
        else:
            x = x[np.newaxis, :]

    return x

In [203]:
def _logpdf(x, mean, prec_U, log_det_cov):
    dev = x - mean
    maha = np.sum(np.square(np.dot(dev, prec_U)), axis=-1)
    return -0.5 * (log_det_cov + maha)

def importance_weight(x, mean_p, cov_p, Pis, means_q, covs_q):
    def logpdf(x, mean=None, cov=1, allow_singular=False):
        dim, mean, cov = _process_parameters(None, mean, cov)
        x = _process_quantiles(x, dim)
        psd = _PSD(cov, allow_singular=allow_singular)
        return _logpdf(x, mean, psd.U, psd.log_pdet)
    w_inv = 0 
    for i in range(Pis.shape[0]):
        w_inv += Pis[i] * np.exp(logpdf(x, mean=means_q[i], cov=covs_q[i]) - logpdf(x, mean=mean_p, cov=cov_p))
        print(np.exp(logpdf(x, mean=means_q[i], cov=covs_q[i]) - logpdf(x, mean=mean_p, cov=cov_p)))
    return 1 / w_inv

In [204]:
def buildPi(K):
    Pi = np.random.dirichlet(([1]*K), 1)[0]
    return Pi

def buildSubModels(mean, cov, K):
    means = np.random.multivariate_normal(mean, cov, K)
    covs = np.repeat(cov[np.newaxis, :, :], K, axis=0)    
    return means, covs

def get_qz(z, Pis, means, covs):
    qz = 0
    for i in range(Pis.shape[0]):
        qz += Pis[i] * multivariate_normal.pdf(z, mean=means[i], cov=covs[i])
        print(Pis[i],multivariate_normal.pdf(z, mean=means[i], cov=covs[i]))
    return qz

def drawFromGMM(Pis, means, covs, N):
    zs = np.empty([N, 512])
    ps = np.empty([N])
    # using numpy multinomial dist
    print(Pis)
    trials = np.random.multinomial(1, Pis, N).tolist()
    #print(trials)
    for i in range(N):
        k = np.nonzero(trials[i])[0].tolist()[0]
        print(Pis[k])
        z = np.random.multivariate_normal(means[k], covs[k], 1)
        zs[i,:] = z
        qz = get_qz(z, Pis, means, covs)
        pz = multivariate_normal.pdf(z, mean=np.zeros(512), cov=np.eye(512))
        #print(qz, pz)
        ps[i] = importance_weight(z, mean_p=np.zeros(512), cov_p=np.eye(512), Pis=Pis, means_q=means, covs_q=covs)
        #ps[i] = pz / qz
        print(pz, qz)
        print(pz / qz)
    return zs, ps

In [205]:
K = 1
N = 1
Pis = buildPi(K)
print(Pis)

[1.]


In [206]:
mean, cov = np.zeros(512), np.eye(512)
means, covs = buildSubModels(mean, np.eye(512)*1, K)
#print(means.shape)
#print(covs.shape)
zs, ps = drawFromGMM(Pis, means, covs, N)
print(ps)

[1.]
1.0
1.0 7.358521887e-315
[1.11997133e+123]
0.0 7.358521887e-315
0.0
[8.92880002e-124]


In [151]:
K = 100
Pis = buildPi(K)
print(Pis)

[0.00908442 0.0032112  0.0036026  0.00409658 0.00277053 0.00570028
 0.00027539 0.00061591 0.00205218 0.0036708  0.01757182 0.01925941
 0.01987497 0.03107719 0.00641927 0.00731989 0.00208208 0.01871404
 0.01008166 0.00643433 0.00697906 0.00429529 0.00082237 0.0042681
 0.00269322 0.00909127 0.00148649 0.00441029 0.01402152 0.00759347
 0.01404936 0.02583599 0.01884285 0.00493303 0.01358098 0.00830726
 0.01078627 0.00090966 0.0018266  0.01430085 0.00345442 0.00501952
 0.00877681 0.01074191 0.00236412 0.0111645  0.00334175 0.00357775
 0.00088995 0.00317169 0.00453651 0.00722514 0.00049082 0.03157998
 0.00139817 0.00157712 0.01277446 0.00644325 0.01299583 0.01776364
 0.00904937 0.00326775 0.00719684 0.00194366 0.01866882 0.01279524
 0.04172823 0.00054822 0.00903395 0.00353267 0.00061126 0.02493707
 0.01353102 0.00817915 0.0350253  0.02022149 0.00894268 0.00582882
 0.02266099 0.00328999 0.0171723  0.00658588 0.02546971 0.02240307
 0.00680101 0.00740738 0.00439192 0.00328136 0.02677662 0.00121

In [152]:
print(zs.shape)

(10, 512)
